In [ ]:
import caffe
caffe.set_mode_gpu()
caffe.set_device(0)
nn = 'ResNet50-Single_Stern'
nf = 'ResNet50-Single-FCN_Stern'

#nn = 'VGG16-Single'
#nf = 'FCN32_posPatchRefining'

it = '4000'

In [ ]:
net = caffe.Net('../data/models/'+nn+'/deploy.prototxt', '../data/models/'+nn+'/snapshots/train_iter_'+it+'.caffemodel', caffe.TEST)
params = ['fc']
#params = ['fc6_', 'fc7_', 'fc8_']
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

for fc in params:
    print('{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape))

In [ ]:
# Load the fully convolutional network to transplant the parameters.
net_full_conv = caffe.Net('../data/models/'+nf+'/deploy.prototxt', 
                          '../data/models/'+nn+'/snapshots/train_iter_'+it+'.caffemodel',
                          caffe.TEST)
params_full_conv = ['fc_conv']
#params_full_conv = ['fc6_conv', 'fc7_conv', 'score_fr_']
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

for conv in params_full_conv:
    print('{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape))

In [ ]:
conv_params['fc_conv'][0].shape

In [ ]:
for pr, pr_conv in zip(params, params_full_conv):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]


In [ ]:
net_full_conv.save('../data/models/'+nf+'/snapshots/classifier_train_iter_'+it+'.caffemodel')